In [1]:
from Cell.morpho_utils import plot_and_save_swcs
from pathlib import Path

2021-08-02 13:30:22,316 numexpr.utils INFO     NumExpr defaulting to 8 threads.


# Plot morphology

In [2]:
swcs = list(Path('swc_own').iterdir())
fig_path = Path('swc_own_fig')
fig_path.mkdir(exist_ok=True)
plot_and_save_swcs(swcs, fig_path)

  0%|          | 0/392 [00:00<?, ?it/s]

2021-08-02 13:30:26,125 root         WARNING  Use neuron_morphology to plot swc_own/2019-05-19cell8.DAT.swc.
2021-08-02 13:36:46,230 root         WARNING  Use neuron_morphology to plot swc_own/2019-07-19cell1.DAT.swc.
2021-08-02 13:51:35,810 root         WARNING  Use neuron_morphology to plot swc_own/2019-10-25cell8.DAT.swc.
2021-08-02 13:52:10,400 root         WARNING  Use neuron_morphology to plot swc_own/2019-10-27cell7.DAT.swc.
2021-08-02 13:55:26,326 root         WARNING  Use neuron_morphology to plot swc_own/2019-11-18cell4.DAT.swc.
2021-08-02 14:04:06,025 root         WARNING  Use neuron_morphology to plot swc_own/20191218cell5.DAT.swc.
